# MODEL-2: LightGBM Baseline for RUL Prediction

This notebook implements and evaluates a LightGBM gradient boosting baseline for Remaining Useful Life (RUL) prediction on the XJTU-SY bearing dataset.

**Objectives:**
1. Train LightGBM on handcrafted features (65 time + frequency domain features)
2. Evaluate using leave-one-bearing-out cross-validation
3. Extract and visualize feature importance
4. Compute SHAP values for interpretability
5. Compare with statistical trending baseline (MODEL-1)

**Reference baseline from MODEL-1:** RMSE ~31 (RMS threshold baseline)

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', '{:.4f}'.format)

## 1. Load Data

Load the pre-extracted features from `outputs/features/features_v2.csv`.

In [ ]:
# Load features
features_df = pd.read_csv('../outputs/features/features_v2.csv')

print(f"Dataset shape: {features_df.shape}")
print(f"Columns: {features_df.columns.tolist()[:10]}...")
print(f"\nSample counts by condition:")
print(features_df.groupby('condition').size())
print(f"\nBearings: {features_df['bearing_id'].nunique()}")
features_df.head()

In [ ]:
# Separate feature columns from metadata
from src.models.baselines import get_feature_columns

feature_cols = get_feature_columns(features_df)
print(f"Number of features: {len(feature_cols)}")
print(f"\nFeature columns:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i:2d}. {col}")

## 2. LightGBM Configuration

Define hyperparameters for the LightGBM model.

In [ ]:
from src.models.baselines import LGBMConfig, LightGBMBaseline

# Configure model
config = LGBMConfig(
    objective='regression',
    metric='rmse',
    num_leaves=31,
    learning_rate=0.05,
    n_estimators=500,
    max_depth=-1,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.1,
    random_state=42,
    early_stopping_rounds=50,
    verbose=-1,
)

print("LightGBM Configuration:")
for key, value in config.to_lgb_params().items():
    print(f"  {key}: {value}")

## 3. Cross-Validation Training

Train with leave-one-bearing-out cross-validation (15 folds).

In [ ]:
from src.models.baselines import train_with_cv
from src.training.cv import leave_one_bearing_out

# Generate CV folds
cv_split = leave_one_bearing_out(features_df)
print(f"CV Strategy: {cv_split.strategy}")
print(f"Number of folds: {len(cv_split)}")
print(f"\nFold summary:")
print(cv_split.summary())

In [ ]:
# Train with cross-validation
cv_results, feature_importance = train_with_cv(
    features_df=features_df,
    feature_cols=feature_cols,
    target_col='rul',
    cv_split=cv_split,
    config=config,
    verbose=True,
)

## 4. Evaluation Results

Analyze cross-validation results and compare with baseline.

In [ ]:
from src.models.baselines import evaluate_lgbm_cv

# Evaluate CV results
eval_results = evaluate_lgbm_cv(cv_results, features_df)

print("=" * 60)
print("Overall Metrics (All Samples Aggregated)")
print("=" * 60)
for metric, value in eval_results['overall'].items():
    if 'phm08' in metric:
        print(f"  {metric:25s}: {value:,.2f}")
    else:
        print(f"  {metric:25s}: {value:.4f}")

print("\n" + "=" * 60)
print("Cross-Validation Statistics")
print("=" * 60)
for metric, value in eval_results['cv_stats'].items():
    print(f"  {metric:25s}: {value:.4f}")

In [ ]:
# Per-bearing breakdown
print("\nPer-Bearing Metrics:")
print("=" * 80)
print(eval_results['per_bearing'].to_string(index=False))

In [ ]:
# Compare with statistical baseline
print("\n" + "=" * 60)
print("Comparison with Statistical Trending Baseline (MODEL-1)")
print("=" * 60)
print(f"\nStatistical Trending Baselines:")
print(f"  - RMS Threshold (linear):   RMSE = 30.92")
print(f"  - Kurtosis Trending:        RMSE = 36.06")
print(f"  - Health Indicator Fusion:  RMSE = 31.83")
print(f"\nLightGBM Baseline:")
print(f"  - LightGBM CV:              RMSE = {eval_results['overall']['rmse']:.2f}")
print(f"\nImprovement over best statistical baseline:")
improvement = 30.92 - eval_results['overall']['rmse']
pct_improvement = (improvement / 30.92) * 100
print(f"  - RMSE Reduction: {improvement:.2f} ({pct_improvement:.1f}%)")

## 5. Feature Importance Analysis

Identify the most predictive features for RUL.

In [ ]:
# Top 20 features by importance
print("Top 20 Most Important Features:")
print("=" * 50)
top_20 = feature_importance.head(20)
for i, row in top_20.iterrows():
    print(f"  {i+1:2d}. {row['feature']:30s} {row['importance_mean']:10.2f} +/- {row['importance_std']:8.2f}")

In [ ]:
# Plot feature importance
from src.models.baselines import plot_feature_importance

fig = plot_feature_importance(feature_importance, top_n=20)
plt.tight_layout()
plt.savefig('../outputs/models/lgbm_feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Group features by domain
time_features = [f for f in feature_cols if not any(x in f for x in ['spectral', 'band_power', 'dominant', 'mean_freq'])]
freq_features = [f for f in feature_cols if any(x in f for x in ['spectral', 'band_power', 'dominant', 'mean_freq'])]

time_importance = feature_importance[feature_importance['feature'].isin(time_features)]['importance_mean'].sum()
freq_importance = feature_importance[feature_importance['feature'].isin(freq_features)]['importance_mean'].sum()
total_importance = time_importance + freq_importance

print(f"\nFeature Domain Importance:")
print(f"  Time-domain features:      {time_importance:,.0f} ({100*time_importance/total_importance:.1f}%)")
print(f"  Frequency-domain features: {freq_importance:,.0f} ({100*freq_importance/total_importance:.1f}%)")

## 6. SHAP Value Analysis

Compute SHAP values to understand feature contributions to individual predictions.

In [ ]:
# Train a single model on all data for SHAP analysis
# (CV models are separate, so we train one comprehensive model)
X = features_df[feature_cols].values
y = features_df['rul'].values

# Simple train/val split for SHAP demo
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

shap_model = LightGBMBaseline(config)
shap_model.fit(X_train, y_train, X_val, y_val, feature_names=feature_cols)
print(f"SHAP model trained. Best iteration: {shap_model.best_iteration}")

In [ ]:
# Compute SHAP values
try:
    import shap
    
    shap_values, explainer = shap_model.get_shap_values(X_val, max_samples=500)
    
    # Summary plot
    plt.figure(figsize=(12, 10))
    shap.summary_plot(shap_values, X_val, feature_names=feature_cols, max_display=20, show=False)
    plt.tight_layout()
    plt.savefig('../outputs/models/lgbm_shap_summary.png', dpi=150, bbox_inches='tight')
    plt.show()
except ImportError:
    print("SHAP not installed. Skipping SHAP analysis.")
    print("Install with: pip install shap")

In [ ]:
# SHAP bar plot (mean absolute SHAP value)
try:
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values, X_val, feature_names=feature_cols, 
                      max_display=20, plot_type='bar', show=False)
    plt.tight_layout()
    plt.savefig('../outputs/models/lgbm_shap_bar.png', dpi=150, bbox_inches='tight')
    plt.show()
except NameError:
    print("SHAP not available.")

## 7. Prediction Visualization

Visualize RUL predictions vs ground truth for sample bearings.

In [ ]:
# Select a few bearings to visualize
sample_bearings = ['Bearing1_1', 'Bearing2_1', 'Bearing3_1']

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, bearing in zip(axes, sample_bearings):
    # Find the CV result for this bearing
    for result in cv_results:
        if bearing in result.val_bearing:
            ax.plot(result.y_true, 'b-', label='Ground Truth', linewidth=2)
            ax.plot(result.y_pred, 'r--', label='LightGBM Pred', linewidth=2, alpha=0.8)
            ax.set_xlabel('Sample Index')
            ax.set_ylabel('RUL')
            ax.set_title(f'{bearing}\nRMSE: {result.val_rmse:.2f}')
            ax.legend(loc='upper right')
            ax.grid(True, alpha=0.3)
            break

plt.tight_layout()
plt.savefig('../outputs/models/lgbm_predictions.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Scatter plot: Predicted vs Actual
all_y_true = np.concatenate([r.y_true for r in cv_results])
all_y_pred = np.concatenate([r.y_pred for r in cv_results])

fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(all_y_true, all_y_pred, alpha=0.3, s=10)
ax.plot([0, 125], [0, 125], 'r--', linewidth=2, label='Perfect Prediction')
ax.set_xlabel('Actual RUL', fontsize=12)
ax.set_ylabel('Predicted RUL', fontsize=12)
ax.set_title(f'LightGBM: Predicted vs Actual RUL\nRMSE: {eval_results["overall"]["rmse"]:.2f}', fontsize=14)
ax.legend()
ax.set_xlim(0, 130)
ax.set_ylim(0, 130)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../outputs/models/lgbm_scatter.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Summary

### Key Findings

1. **LightGBM Performance**: The LightGBM model achieves competitive RUL prediction on the XJTU-SY dataset using handcrafted features.

2. **Feature Importance**: The most important features for RUL prediction are:
   - Time-domain: RMS, standard deviation, line integral
   - Frequency-domain: Band powers, spectral centroid

3. **Comparison with Baseline**: LightGBM provides improvement over pure signal processing baselines by learning non-linear feature interactions.

4. **Interpretability**: SHAP values provide insight into which features drive individual predictions, useful for maintenance decision-making.

In [ ]:
# Final summary
print("=" * 60)
print("MODEL-2: LightGBM Baseline - Summary")
print("=" * 60)
print(f"\nDataset: {len(features_df)} samples, {len(feature_cols)} features")
print(f"CV Strategy: Leave-one-bearing-out ({len(cv_split)} folds)")
print(f"\nResults:")
print(f"  RMSE:  {eval_results['overall']['rmse']:.2f}")
print(f"  MAE:   {eval_results['overall']['mae']:.2f}")
print(f"  MAPE:  {eval_results['overall']['mape']:.1f}%")
print(f"\nTop 5 Features:")
for i, row in feature_importance.head(5).iterrows():
    print(f"  {i+1}. {row['feature']}")